# Challenge SD207 - 2017
*<p>Author: Pengfei MI, Rui SONG</p>*
*<p>Date: 06/06/2017</p>*

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import librosa

In [ ]:
# Preprocessing